In [1]:
import sys
import os
import indra_gpt
from tqdm import tqdm
import pickle
import json
from pathlib import Path
import datetime


In [2]:
data_path = Path.cwd() / "indra_gpt" / "resources" / "indra_benchmark_corpus_all_correct.json"
data = json.load(open(data_path))
inputs = [(ev.get('text', ""), ev.get('pmid', '')) for stmt in data for ev in stmt['evidence']]


In [3]:
from indra_gpt.clients.llm_client import LLMClient

model_config = {
    "openai": {
        "api_key": os.getenv("OPENAI_API_KEY"),
        "model": "gpt-4o-mini"
    },
    "anthropic": {
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "model": "claude-3-5-haiku-latest"
    },
    "deepseek": {
        "api_key": os.getenv("DEEPSEEK_API_KEY"),
        "model": "deepseek-chat"
    },
    "ollama": {
        "api_key": None, 
        "model": "llama3.2:latest", 
        "api_base": "http://localhost:11434"
    }
}

api_provider = "openai"
llm_client = LLMClient(
    custom_llm_provider=api_provider,
    api_key = model_config[api_provider]["api_key"],
    model = model_config[api_provider]["model"],
    response_format="json"
)


In [4]:
from indra_gpt.preprocessors.preprocessor import Preprocessor
from indra_gpt.postprocess.postprocessor import PostProcessor
from indra_gpt.postprocess.json_to_indra import JSONToINDRAConverter
from indra_gpt.postprocess.preassemble_indra import INDRAPreassembler
from indra_gpt.extractors.end_to_end.end_to_end_extractor import EndToEndExtractor


# Preprocessor
rec_logic_preprocessor = Preprocessor(llm_client, 
                            preprocessing_method="rec_logic_parser")
rec_relation_preprocessor = Preprocessor(llm_client,
                            preprocessing_method="rec_relation_parser")
# Extractor
extractor = EndToEndExtractor(llm_client=llm_client, num_history_examples=2)
# Postprocessor
json_to_indra_converter = JSONToINDRAConverter()
preassembler = INDRAPreassembler()
postprocessor = PostProcessor(json_to_indra_converter, preassembler)


INFO: [2025-07-10 12:13:09] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.


In [5]:
import pickle
input_data = pickle.load(open(Path.cwd() / "indra_gpt" / "resources" / "test_set.pkl", "rb"))


In [ ]:
# from indra_gpt.extractors.recursive.recursive_extractor import recursive_extract, get_indra_stmts_json_list

# recursive_extract('Of note, Tiam1 directly interacts with activated Ras through a Ras binding domain and has also been identified as a target of Wnt signaling [XREF_BIBR], suggesting two potential areas of overlap with known OPN regulators for further study.', llm_client)


12:40:08 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:40:08] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:40:10] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:40:10 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO: [2025-07-10 12:40:10] LiteLLM - Wrapper: Completed Call, calling success_handler
12:40:10 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:40:10] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:40:57 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:40:57] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:40:58] httpx - HTTP Request: POST https://api.openai.com

IndraStatements(regulate_activity_list=[], modification_list=[], self_modification_list=[], activeform_list=[], gef_list=[Gef(evidence=[], supports=[], supported_by=[], uuid=None, type='Gef', gef=Agent(name='Tiam1', db_refs=None, mods=[], bound_conditions=[], mutations=[], activity=None, location=None, sbo=None), ras=Agent(name='Ras', db_refs=None, mods=[], bound_conditions=[], mutations=[], activity=None, location=None, sbo=None))], gap_list=[], complex_list=[], translocation_list=[], regulate_amount_list=[], conversion_list=[])

In [ ]:
from indra_gpt.extractors.recursive.recursive_extractor import recursive_extract, get_indra_stmts_json_list

all_indra_stmts = []
for input in tqdm(input_data, desc="Processing inputs", unit="input"):
    text = input.get("text", "")
    pmid = input.get("pmid", "")
    try:
        result = recursive_extract(text, llm_client)
        stmts_json = get_indra_stmts_json_list(result)
        indra_stmts = postprocessor.convert_to_indra_stmts(json.dumps(stmts_json), text, pmid)
    except Exception as e:
        print(f"Error processing {pmid}: {e}")
        indra_stmts = []
    all_indra_stmts.extend(indra_stmts)


Processing inputs:   0%|          | 0/100 [00:00<?, ?input/s]12:13:09 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:13:09] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:13:11] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:13:11 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO: [2025-07-10 12:13:11] LiteLLM - Wrapper: Completed Call, calling success_handler
12:13:11 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:13:11] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:13:11 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:13:11 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: o

[Final ValidationError] while constructing BoundCondition at regulate_amount_list[0].obj.bound_conditions[0]:
1 validation error for BoundCondition
agent
  Input should be a valid dictionary or instance of Agent [type=model_type, input_value='PDGF', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type


INFO: [2025-07-10 12:14:52] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:14:52 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO: [2025-07-10 12:14:52] LiteLLM - Wrapper: Completed Call, calling success_handler
12:14:52 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:14:52] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:14:52 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:14:52] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:14:52 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:14:52] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
12:14:52 - LiteLLM:INFO: cost_calculator.py:

[Final ValidationError] while constructing Evidence at regulate_amount_list[0].evidence[0]:
1 validation error for Evidence
annotations
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type


INFO: [2025-07-10 12:16:14] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:16:14 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO: [2025-07-10 12:16:14] LiteLLM - Wrapper: Completed Call, calling success_handler
12:16:14 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:16:14] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:16:14 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:16:14] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:16:14 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:16:14] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
12:16:14 - LiteLLM:INFO: cost_calculator.py:

...

[Final ValidationError] while constructing Evidence at gef_list[0].evidence[0]:
1 validation error for Evidence
annotations
  Input should be a valid dictionary [type=dict_type, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type


INFO: [2025-07-10 12:29:33] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
12:29:33 - LiteLLM:INFO: utils.py:1213 - Wrapper: Completed Call, calling success_handler
INFO: [2025-07-10 12:29:33] LiteLLM - Wrapper: Completed Call, calling success_handler
12:29:33 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:29:33] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:29:33 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:29:33] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:29:33 - LiteLLM:INFO: utils.py:2931 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: [2025-07-10 12:29:33] LiteLLM - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
12:29:33 - LiteLLM:INFO: cost_calculator.py:

[JSONError] at IndraStatements:
Expecting property name enclosed in double quotes: line 14 column 1 (char 296)
Error processing 21859891: 'NoneType' object has no attribute 'model_dump'



12:29:38 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:29:38] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18


12:38:43 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:38:43] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:38:43 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:38:43] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:38:45 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:38:45] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:38:45 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
INFO: [2025-07-10 12:38:45] LiteLLM - selected model name for cost calculation: openai/gpt-4o-mini-2024-07-18
12:38:46 - LiteLLM:INFO: cost_calculator.py:655 - selected model name fo

In [8]:
default_grounded_stmts = postprocessor.default_grounder(all_indra_stmts)


In [9]:
preassembled_stmts = postprocessor.preassemble(default_grounded_stmts)


INFO: [2025-07-10 12:29:38] indra_gpt.postprocess.preassemble_indra - Running preassembly pipeline on 104 statements...
INFO: [2025-07-10 12:29:38] indra.pipeline.pipeline - Running the pipeline
INFO: [2025-07-10 12:29:38] indra.pipeline.pipeline - Calling filter_no_hypothesis
INFO: [2025-07-10 12:29:38] indra.tools.assemble_corpus - Filtering 104 statements to no hypothesis...
INFO: [2025-07-10 12:29:38] indra.tools.assemble_corpus - 104 statements after filter...
INFO: [2025-07-10 12:29:38] indra.pipeline.pipeline - Calling filter_no_negated
INFO: [2025-07-10 12:29:38] indra.tools.assemble_corpus - Filtering 104 statements to not negated...
INFO: [2025-07-10 12:29:38] indra.tools.assemble_corpus - 104 statements after filter...
INFO: [2025-07-10 12:29:38] indra.pipeline.pipeline - Calling custom_ground_statements
INFO: [2025-07-10 12:29:38] indra_gpt.postprocess.preassemble_indra - Applying GILDA grounding to 104 statements...
INFO: [2025-07-10 12:29:38] indra.ontology.bio.ontology -

In [13]:
output_path = Path.cwd() / "output" / "recursive_extraction"
output_path.mkdir(parents=True, exist_ok=True)
date_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

pickle.dump(preassembled_stmts, open(output_path / f"preassembled_stmts_{date_str}.pkl", "wb"))


In [12]:
len(preassembled_stmts)


45